In [1]:
import numpy as np
import pandas as pd
import torch
from scipy.linalg import toeplitz
from scipy.stats import norm

import matplotlib.pyplot as plt
from tableone import TableOne
from sksurv.nonparametric import kaplan_meier_estimator

from utils import data_processing, visualization
from utils.simulations import *
from execute import surv_hivae
#from utils.metrics import log_rank, cox_estimation

In [2]:
n_samples = 600
n_features_bytype = 4
n_features_multiplier = 3 

In [3]:
beta_features = np.concatenate([weights_sparse_exp(n_features_bytype,3),weights_sparse_exp(n_features_bytype,3),
                       weights_sparse_exp(n_features_bytype,3)])
treatment_effect = 0

## Independent

In [4]:
control, treated, types = simulation(beta_features, treatment_effect , n_samples, independent = True, surv_type = "surv_weibull",
                                     n_features_multiplier = 3, nnz = 3 , p_treated = 0.5, a_T = 2, a_C = 1, lamb_C = 0.5, 
                                     data_types_create = True)
control = control.drop(columns='treatment')
treated = treated.drop(columns='treatment')

In [5]:
print(np.mean(control['censor']),np.mean(treated['censor']))

0.15081967213114755 0.13220338983050847


## Dependent

In [6]:
control, treated, types = simulation(beta_features, treatment_effect , n_samples, independent = False, surv_type = "surv_weibull",
                                     n_features_multiplier = 3, nnz = 3 , p_treated = 0.5, a_T = 2, a_C = 1, lamb_C = 0.15, 
                                     data_types_create = True)
control = control.drop(columns='treatment')
treated = treated.drop(columns='treatment')

In [7]:
print(np.mean(control['censor']),np.mean(treated['censor']))

0.13953488372093023 0.12709030100334448


## Compute expected power / level via Schoenfeld formula

$$D = \frac{(\Phi^{-1}(\beta)+\Phi^{-1}(1-\alpha))^2}{P_{cont}(1 - P_{cont}) log^2(\Delta)}$$
where 
- $D$ is the number of deaths
- $\alpha$ is the level
- $\beta$ is the power
- $P_{cont}$ is the proportion of patients in the control arm
- $\Delta$ is the hazard ratio

D

In [14]:
data = pd.concat([control,treated],ignore_index=True)
D = np.sum(1-data['censor'])
print(D)

520


In [15]:
alpha = 0.05
p_treated = 0.5
treatment_effect = 0.5
expected_power = norm.cdf(np.sqrt( D * p_treated * (1 - p_treated) * (treatment_effect)**2 ) - norm.ppf(1 - alpha/2))

In [16]:
expected_power

0.999908323558307